In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/instquality/")

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/instquality/")

import warnings
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

from functions.spec import run_unified_regularized_regression, filter_cols
from functions.utils import check_split_year
from functions.summary import get_benchmark_stats

### Lagged incremental signal regressions

In [3]:
# load data
df = pd.read_pickle("saved/df.dat")

# define columns
id_cols = ["country", "year", "iso_code_1", "iso_code_2", "region"]
exclude = id_cols + [c for c in df.columns if c.startswith("tgt_")]
macro_cols = filter_cols([c for c in df.columns if c.startswith("wb_") and not c.startswith("wb_iq_") and c not in exclude], [])
iq_cols = filter_cols([c for c in df.columns if (c.startswith("wb_iq_") or (not c.startswith("wb_") and c not in exclude))], [])
macro_cols += ["tgt_spread"]

# define models to run
models = ['lasso', 'ridge', 'elastic']

# iterate over split shares
split_shares = [0.75, 0.8, 0.85]
for split_share in split_shares:
    print("\n" + "=" * 100)
    print(f"Runnning {split_share*100:.0f}-{(1-split_share)*100:.0f} split...")

    # run for each horizon
    for h in range(1, 11):

        # get split year and benchmarks
        y_name = f"tgt_spread_t{h}"
        split_year = check_split_year(df, y_name, split_share)
        if split_share != 0.75:
            name_prefix = f"signal_t{h}_{split_share*100:.0f}"
        else:
            name_prefix = f"signal_t{h}"
        benchmark_df = get_benchmark_stats(df, split_share=split_share)
        h_benchmark = benchmark_df[benchmark_df["horizon"] == h].iloc[0]
        r2_benchmark = h_benchmark["R2_test"]
        rmse_benchmark = h_benchmark["RMSE_test"]

        # run spec for models
        print("\n" + "=" * 100)
        print(f"Running forecast horizon: t+{h}")
        print(f"Split year for {split_share*100:.0f}-{(1-split_share)*100:.0f} split: {split_year}")
        all_horizon_results = {}
        run_unified_regularized_regression(
            name_prefix=name_prefix,
            data_path="saved/df.dat",
            macro_cols=macro_cols,
            iq_cols=iq_cols,
            use_clustering=True,
            corr_threshold=0.9,
            y_name=y_name,
            spread_col="tgt_spread",
            year_col="year",
            split_year=split_year,
            model_types=models,
            agnostic=False,
            param_factors=np.logspace(-3, 10, 25),
            l1_ratios=[0.5],
            use_expanding_cv=True,
            min_train_years=8,
            cv_stride=2,
            r2_benchmark=r2_benchmark,
            rmse_benchmark=rmse_benchmark,
            early_stop_threshold=0.10,
            save_results=True,
            output_dir="specs",
            n_jobs=-1,
            multicollinearity_check=False,
            perform_interpretability=True
        )


Runnning 75-25 split...

Running forecast horizon: t+1
Split year for 75-25 split: 2014

UNIFIED REGULARIZED REGRESSION: SIGNAL_T1
Specification: β=1 (fixed at mean reversion)
Models: LASSO, RIDGE, ELASTIC
Elastic Net L1 ratios: [0.5]
Feature clustering: ON (threshold=0.9)
Statistical inference: SHAP values, Stability Selection (Lasso/Elastic Net only)
R² convergence: Stop after 3 consecutive changes < 0.0001

Train: 1960-2014 (1564 obs)
Test:  2015-2023 (534 obs)

β=1 specification active
After removing NaN: 1564 train, 534 test obs

Benchmark metrics:
  R² (test): 0.9074
  RMSE (test): 1.2728

Clustering variants of same variables...
  Original features: 972
  After clustering: 640
  Number of clusters: 640

Features used in models: 640


--------------------------------------------------------------------------------
RUNNING LASSO
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal paramete

,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.001000,0.000483,0.952514,0.489696,2.988004,565,-11.909514,0.000000e+00
1,0.003481,0.001682,0.945931,0.531183,2.863972,473,-12.331897,0.000000e+00
2,0.012115,0.005855,0.932586,0.614423,2.597301,351,-11.398731,0.000000e+00
3,0.042170,0.020380,0.910313,0.823581,1.756868,204,-5.101179,3.375450e-07
4,0.146780,0.070938,0.864961,0.879063,1.454611,90,-2.980710,2.875811e-03
5,0.510897,0.246913,0.816900,0.902750,1.304405,13,-1.227936,2.194709e-01
6,1.778279,0.859430,0.758771,0.908984,1.261905,1,0.997312,3.186132e-01



Best LASSO:
  Parameter: 0.859430
  R² (test): 0.9090
  ΔR² vs benchmark: +0.16 p.p.
  RMSE (test): 1.2619
  ΔRMSE vs benchmark: -0.0109
  Active vars: 1
  Active variables: wb_fp_cpi_totl_zg
  DM test: stat=1.00, p=0.319

Skipping interpretability analysis: no significant incremental value over mean reversion benchmark (DM p=0.319)

Interpretation: Macro/IQ variables do not provide statistically significant
incremental predictive power beyond the mean reversion benchmark.


--------------------------------------------------------------------------------
RUNNING RIDGE
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 24297.810658

Early stopping: R² converged after 3 iterations.



,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.001000,2.429781e+01,0.938644,0.579978,2.710833,640,-11.976861,0.000000e+00
1,0.003481,8.457340e+01,0.927629,0.686802,2.340864,640,-9.936409,0.000000e+00
2,0.012115,2.943747e+02,0.909781,0.800096,1.870154,640,-7.046383,1.836309e-12
3,0.042170,1.024630e+03,0.882594,0.861946,1.554145,640,-5.132073,2.865685e-07
4,0.146780,3.566431e+03,0.837401,0.888385,1.397424,640,-4.222864,2.412171e-05
5,0.510897,1.241368e+04,0.748635,0.900840,1.317152,640,-2.813241,4.904484e-03
6,1.778279,4.320830e+04,0.636128,0.906704,1.277613,640,-0.536661,5.915021e-01
7,6.189658,1.503951e+05,0.564418,0.908355,1.266254,640,0.887477,3.748220e-01
8,21.544347,5.234805e+05,0.535773,0.908522,1.265099,640,1.029974,3.030224e-01
9,74.989421,1.822079e+06,0.526517,0.908478,1.265403,640,0.996816,3.188536e-01



Best RIDGE:
  Parameter: 523480.461736
  R² (test): 0.9085
  ΔR² vs benchmark: +0.11 p.p.
  RMSE (test): 1.2651
  ΔRMSE vs benchmark: -0.0077
  Active vars: 640
  DM test: stat=1.03, p=0.303

Skipping interpretability analysis: no significant incremental value over mean reversion benchmark (DM p=0.303)

Interpretation: Macro/IQ variables do not provide statistically significant
incremental predictive power beyond the mean reversion benchmark.


--------------------------------------------------------------------------------
RUNNING ELASTIC (L1=0.5)
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 0.885867

Stopping: model has 0 active variables at factor 6.189658188912603



...


Best ELASTIC (L1=0.5):
  Parameter: 6.543189
  R² (test): 0.4958
  ΔR² vs benchmark: +8.00 p.p.
  RMSE (test): 2.9483
  ΔRMSE vs benchmark: -0.2254
  Active vars: 6
  Active variables: wb_fp_cpi_totl_zg_ma3, wb_fp_cpi_totl_zg_ma10, wb_fp_cpi_totl_zg, wb_ny_gdp_defl_kd_zg, wb_ny_gdp_defl_kd_zg_ma3, wb_ny_gdp_defl_kd_zg_t-1
  DM test: stat=5.33, p=0.000


INTERPRETABILITY ANALYSIS: WHICH VARIABLES DRIVE INCREMENTAL PREDICTIVE POWER?
(ELASTIC (L1=0.5))

METHOD 1: SHAP VALUES (Feature Attribution)

Computing SHAP values...

Top 2 core variables by SHAP importance:



,core_variable,variants,shap_importance,shap_importance_pct,n_variants
88,wb_fp_cpi_totl_zg,"[wb_fp_cpi_totl_zg_ma3, wb_fp_cpi_totl_zg, wb_...",0.019057,55.37736,6
93,wb_ny_gdp_defl_kd_zg,"[wb_ny_gdp_defl_kd_zg_ma3, wb_ny_gdp_defl_kd_z...",0.015356,44.62264,8



METHOD 2: STABILITY SELECTION

Running stability selection (100 iterations)...

Core variables with stable variants (aggregated from 6 stable features):



,core_variable,variants,selection_frequency,mean_coef,std_coef,n_variants
0,wb_fp_cpi_totl_zg,"[wb_fp_cpi_totl_zg, wb_fp_cpi_totl_zg_ma10, wb...",0.850000,-0.436395,0.060777,3
1,wb_ny_gdp_defl_kd_zg,"[wb_ny_gdp_defl_kd_zg_delta3, wb_ny_gdp_defl_k...",0.843333,-0.283276,0.067648,3



COMPLETED IN 187.0s



### Lagged priced-in level regressions

In [ ]:
# load data
df = pd.read_pickle("saved/df.dat")

# define columns
id_cols = ["country","year","iso_code_1","iso_code_2","region"]
exclude = id_cols + [c for c in df.columns if c.startswith("tgt_")]
macro_cols = filter_cols([c for c in df.columns if c.startswith("wb_") and not c.startswith("wb_iq_") and c not in exclude], [])
iq_cols = filter_cols([c for c in df.columns if (c.startswith("wb_iq_") or (not c.startswith("wb_") and c not in exclude))], [])

# define models to run
models = ['lasso', 'ridge', 'elastic']

# iterate over split shares
split_shares = [0.75, 0.8, 0.85]
for split_share in split_shares:
    print("\n" + "=" * 100)
    print(f"Runnning {split_share*100:.0f}-{(1-split_share)*100:.0f} split...")

    # run models for each horizon
    for h in range(1,11):

        # get split year for horizon
        y_name = f"tgt_spread_t{h}"
        split_year = check_split_year(df, y_name, split_share)
        if split_share != 0.75:
            name_prefix = f"levels_t{h}_{split_share*100:.0f}"
        else:
            name_prefix = f"levels_t{h}"

        # run spec for models
        print("\n" + "=" * 100)
        print(f"Running forecast horizon: t+{h}")
        print(f"Split year for {split_share*100:.0f}-{(1-split_share)*100:.0f} split: {split_year}")
        run_unified_regularized_regression(
            name_prefix=name_prefix,
            data_path="saved/df.dat",
            macro_cols=macro_cols,
            iq_cols=iq_cols,
            use_clustering=True,
            corr_threshold=0.9,
            y_name=y_name,
            spread_col="tgt_spread",
            year_col="year",
            split_year=split_year,
            model_types=models,
            agnostic=True,
            param_factors=np.logspace(-2,5,15),
            l1_ratios=[0.5],
            use_expanding_cv=True,
            min_train_years=8,
            cv_stride=2,
            r2_benchmark=None,
            rmse_benchmark=None,
            early_stop_threshold=0.10,
            save_results=True,
            output_dir="specs",
            n_jobs=-1,
            multicollinearity_check=False,
            perform_interpretability=True
        )


Runnning 75-25 split...

Running forecast horizon: t+1
Split year for 75-25 split: 2014

UNIFIED REGULARIZED REGRESSION: LEVELS_T1
Specification: Agnostic (β unrestricted)
Models: ELASTIC
Elastic Net L1 ratios: [0.5]
Feature clustering: ON (threshold=0.9)
Statistical inference: SHAP values, Stability Selection (Lasso/Elastic Net only)
R² convergence: Stop after 3 consecutive changes < 0.0001

Train: 1960-2014 (1642 obs)
Test:  2015-2023 (541 obs)

Clustering variants of same variables...
  Original features: 972
  After clustering: 645
  Number of clusters: 645

Features used in models: 645


--------------------------------------------------------------------------------
RUNNING ELASTIC (L1=0.5)
--------------------------------------------------------------------------------

Selecting optimal parameter via expanding window CV...
Optimal parameter: 2.976351

Stopping: model has 0 active variables at factor 3.1622776601683795



,factor,param,R²_train,R²_test,RMSE_test,Active_vars,DM_stat,DM_p
0,0.010000,0.029764,0.912076,0.453278,3.098579,350,NaN,NaN
1,0.031623,0.094120,0.844922,0.498013,2.969107,237,NaN,NaN
2,0.100000,0.297635,0.675938,0.416557,3.200947,112,NaN,NaN
3,0.316228,0.941205,0.483757,0.366866,3.334473,44,NaN,NaN
4,1.000000,2.976351,0.269172,0.207952,3.729538,23,NaN,NaN



Best ELASTIC (L1=0.5):
  Parameter: 0.094120
  R² (test): 0.4980
  RMSE (test): 2.9691
  Active vars: 237


INTERPRETABILITY ANALYSIS: WHICH VARIABLES ARE PRICED IN?
(ELASTIC (L1=0.5))

METHOD 1: SHAP VALUES (Feature Attribution)

Computing SHAP values...

Top 20 core variables by SHAP importance:



,core_variable,variants,shap_importance,shap_importance_pct,n_variants
95,wb_ny_gdp_mktp_kd_zg,"[wb_ny_gdp_mktp_kd_zg_ma10, wb_ny_gdp_mktp_kd_...",1.850665,8.897891,9
9,efw_2a_judicial_independence,"[efw_2a_judicial_independence_t-5, efw_2a_judi...",1.004954,4.831761,5
74,ief_monetary_freedom,"[ief_monetary_freedom, ief_monetary_freedom_ma...",0.821779,3.951066,7
2,efw_1b_transfers_and_subsidies,"[efw_1b_transfers_and_subsidies_ma5, efw_1b_tr...",0.755358,3.631717,6
13,efw_2e_legal_integrity,"[efw_2e_legal_integrity_ma10, efw_2e_legal_int...",0.692371,3.328881,6
97,wb_ny_gdp_pcap_kd,"[wb_ny_gdp_pcap_kd, wb_ny_gdp_pcap_kd_delta3, ...",0.553284,2.660158,3
19,efw_3b_standard_deviation_of_inflation,"[efw_3b_standard_deviation_of_inflation_ma10, ...",0.471195,2.265476,8
16,efw_2h_police_and_crime,"[efw_2h_police_and_crime_t-3, efw_2h_police_an...",0.467821,2.249257,4
85,pts_pts_a,"[pts_pts_a_ma10, pts_pts_a, pts_pts_a_delta, p...",0.421205,2.025130,7
58,efw_ie_state_ownership,"[efw_ie_state_ownership_ma10, efw_ie_state_own...",0.408701,1.965010,7



METHOD 2: STABILITY SELECTION

Running stability selection (100 iterations)...

Core variables with stable variants (aggregated from 130 stable features):



,core_variable,variants,selection_frequency,mean_coef,std_coef,n_variants
1,efw_1b_transfers_and_subsidies,"[efw_1b_transfers_and_subsidies_ma5, efw_1b_tr...",1.000000,0.239347,0.074021,3
58,wb_gc_dod_totl_gd_zs,"[wb_gc_dod_totl_gd_zs_t-1, wb_gc_dod_totl_gd_z...",1.000000,-0.185810,0.062025,2
64,wb_ny_gdp_pcap_kd,[wb_ny_gdp_pcap_kd],1.000000,-0.301262,0.108617,1
36,efw_5ci_regulatory_burden,"[efw_5ci_regulatory_burden_t-3, efw_5ci_regula...",1.000000,0.144620,0.042523,2
8,efw_2f_contracts,[efw_2f_contracts_delta3],1.000000,-0.215635,0.046231,1
7,efw_2e_legal_integrity,"[efw_2e_legal_integrity_t-1, efw_2e_legal_inte...",1.000000,-0.331201,0.087635,2
52,p5d_durable,[p5d_durable],1.000000,-0.335176,0.055965,1
62,wb_ny_gdp_minr_rt_zs,[wb_ny_gdp_minr_rt_zs_ma10],1.000000,0.208723,0.056862,1
40,efw_5dii_business_permits,[efw_5dii_business_permits_t-5],1.000000,-0.238661,0.053060,1
56,wb_bn_cab_xoka_gd_zs,"[wb_bn_cab_xoka_gd_zs_ma5, wb_bn_cab_xoka_gd_z...",0.993333,-0.177026,0.069874,3



COMPLETED IN 96.2s

